# EDA notebook -- scratchbook to play & understand the data
### Feel free to mess around!

In [153]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
from pylab import rcParams
rcParams['figure.figsize'] = 10, 6
display(HTML("<style>.container { width:95% !important; }</style>"))

In [154]:
root = "data/anime-recommendations-database/"
root2 = "data/anime-data-score-staff-synopsis-and-genre/"
anime_data = pd.read_csv(root + "anime.csv")

FileNotFoundError: [Errno 2] File data/anime-recommendations-database/anime.csv does not exist: 'data/anime-recommendations-database/anime.csv'

In [ ]:
print (anime_data.shape)
anime_data.head(10)

In [ ]:
rating_data = pd.read_csv(root + "rating.csv")

In [ ]:
print (rating_data.shape)
rating_data.head(10)

In [ ]:
plt.plot(rating_data.rating.value_counts())

In [155]:
# Seems like there are a ton of -1 ratings (where people didn't rate). May be worthwhile to delete all of them (>1.4M data) 
rating_data.rating.value_counts().plot(kind='bar')
plt.show()

NameError: name 'rating_data' is not defined

In [ ]:
# lol, the anime with the most ratings is deathnote
rating_data.anime_id.value_counts()[:50].plot(kind='bar')
plt.show()

In [ ]:
top_50_frequently_rated = list(rating_data.anime_id.value_counts()[:25].keys())

In [ ]:
anime_data[anime_data['anime_id'].isin(top_50_frequently_rated)]

In [ ]:
new_df = pd.read_csv(root2 + "dataanime.csv")

In [ ]:
print (new_df.shape)
new_df.head()

In [ ]:
new_df = pd.read_csv("data/tidy_anime.csv")

In [ ]:
print(new_df.shape)
new_df.head()

In [156]:
new_df.columns.tolist()

NameError: name 'new_df' is not defined

In [ ]:
desired_cols = ['animeID', 'title_english', 'type', 'source', 'producers', 'genre', 'studio',
               'episodes', 'premiered', 'rating', 'score', 'scored_by', 'rank', 'popularity',
               'members', 'favorites', 'synopsis', 'background']
new_df[desired_cols].head()

# Join the datasets, clean

* Rename rating column (duplicated)
* Join anime_data and rating_data on anime_id
* Remove all ratings of -1

In [ ]:
anime_data = anime_data.rename(columns={"rating": "overall_rating"})
anime_data.head()

In [ ]:
rating_data = rating_data.rename(columns={"rating": "user_rating"})
rating_data.head()

In [ ]:
merged_df = pd.merge(anime_data, rating_data, on='anime_id')
merged_df.head()

In [ ]:
out_df = pd.read_csv('data/relevant_output_data.csv')
out_df = out_df.drop(columns=['Unnamed: 0'])
out_df.head()

In [ ]:
gen_df = pd.read_csv('data/model_training_data.csv')
gen_df = gen_df.drop(columns=['Unnamed: 0'])
gen_df.head()

In [157]:
def std_z(x, avg, n):
    return (x-avg)/n

def avg_std_z(xs, avg):
    sum = 0
    for x in xs:
        sum += (x-avg)**2
    return (sum/len(xs))**(1/2)

def avg(vals):
    if(len(vals) == 1):
        return vals[0]
    return sum(vals)/len(vals)

In [158]:
# from data_modeling EXAMPLE 1 (Cosine, un-normalized), ['Attack on Titan'] input,
# ['Sword Art Online', 'Dragon Ball Z', 'Code Geass: Lelouch of the Rebellion R2', 
#  'Death Note', 'One Punch Man'] output
in_list = ['Attack on Titan']
insyn_list = list(out_df[out_df['title_english'].isin(in_list)]['synopsis'])
for syn in insyn_list:
    print(syn[:300]+"...")
out_list = ['Sword Art Online', 'Dragon Ball Z', 'Code Geass: Lelouch of the Rebellion R2', 'Death Note', 'One Punch Man']
outsyn_list = []
for t in out_list:
    outsyn_list.append(list(out_df[out_df['title_english'] == t]['synopsis'])[0])
for syn in outsyn_list:
    print('>> \t%s'%(syn[:300]+"..."))

Centuries ago, mankind was slaughtered to near extinction by monstrous humanoid creatures called titans, forcing humans to hide in fear behind enormous concentric walls. What makes these giants truly terrifying is that their taste for human flesh is not born out of hunger but what appears to be out ...
>> 	In the year 2022, virtual reality has progressed by leaps and bounds, and a massive online role-playing game called Sword Art Online (SAO) is launched. With the aid of "NerveGear" technology, players can control their avatars within the game using nothing but their own thoughts. Kazuto Kirigaya, nic...
>> 	Five years after winning the World Martial Arts tournament, Gokuu is now living a peaceful life with his wife and son. This changes, however, with the arrival of a mysterious enemy named Raditz who presents himself as Gokuu's long-lost brother. He reveals that Gokuu is a warrior from the once powerf...
>> 	One year has passed since the Black Rebellion, a failed uprising against the

In [159]:
# SCORED_BY
# from data_modeling EXAMPLE 1 (Cosine, un-normalized), ['Attack on Titan'] input,
# ['Sword Art Online', 'Dragon Ball Z', 'Code Geass: Lelouch of the Rebellion R2', 
#  'Death Note', 'One Punch Man'] output
in_list = ['Attack on Titan']
insyn_list = list(gen_df[gen_df['title_english'].isin(in_list)]['scored_by'])
for syn in insyn_list:
    print(syn)
out_list = ['Sword Art Online', 'Dragon Ball Z', 'Code Geass: Lelouch of the Rebellion R2', 'Death Note', 'One Punch Man']
outsyn_list = []
for t in out_list:
    outsyn_list.append(list(gen_df[gen_df['title_english'] == t]['scored_by'])[0])
sumz = 0
for syn in outsyn_list:
    z = (std_z(syn, avg(insyn_list), len(outsyn_list)))
    sumz += abs(z)
    print(">> \t%s, \t%s"%(syn,z))

print("AVG ST.Z: %s"%(sumz/len(outsyn_list)))
print("AVG SQ ST.Z: %s"%(avg_std_z(outsyn_list,avg(insyn_list))))

1038161
>> 	1007035, 	-6225.2
>> 	402779, 	-127076.4
>> 	591272, 	-89377.8
>> 	1107955, 	13958.8
>> 	806647, 	-46302.8
AVG ST.Z: 56588.2
AVG SQ ST.Z: 364104.0584099551


In [160]:
# POPULARITY
# from data_modeling EXAMPLE 1 (Cosine, un-normalized), ['Attack on Titan'] input,
# ['Sword Art Online', 'Dragon Ball Z', 'Code Geass: Lelouch of the Rebellion R2', 
#  'Death Note', 'One Punch Man'] output
in_list = ['Attack on Titan']
insyn_list = list(gen_df[gen_df['title_english'].isin(in_list)]['popularity'])
for syn in insyn_list:
    print(syn)
out_list = ['Sword Art Online', 'Dragon Ball Z', 'Code Geass: Lelouch of the Rebellion R2', 'Death Note', 'One Punch Man']
outsyn_list = []
for t in out_list:
    outsyn_list.append(list(gen_df[gen_df['title_english'] == t]['popularity'])[0])
sumz = 0
for syn in outsyn_list:
    z = (std_z(syn, avg(insyn_list), len(outsyn_list)))
    sumz += abs(z)
    print(">> \t%s, \t%s"%(syn,z))

print("AVG ST.Z: %s"%(sumz/len(outsyn_list)))
print("AVG SQ ST.Z: %s"%(avg_std_z(outsyn_list,avg(insyn_list))))

2
>> 	3, 	0.2
>> 	71, 	13.8
>> 	24, 	4.4
>> 	1, 	-0.2
>> 	5, 	0.6
AVG ST.Z: 3.84
AVG SQ ST.Z: 32.42221460665511


In [161]:
# EPISODES
# from data_modeling EXAMPLE 1 (Cosine, un-normalized), ['Attack on Titan'] input,
# ['Sword Art Online', 'Dragon Ball Z', 'Code Geass: Lelouch of the Rebellion R2', 
#  'Death Note', 'One Punch Man'] output
in_list = ['Attack on Titan']
insyn_list = list(gen_df[gen_df['title_english'].isin(in_list)]['episodes'])
for syn in insyn_list:
    print(syn)
out_list = ['Sword Art Online', 'Dragon Ball Z', 'Code Geass: Lelouch of the Rebellion R2', 'Death Note', 'One Punch Man']
outsyn_list = []
for t in out_list:
    outsyn_list.append(list(gen_df[gen_df['title_english'] == t]['episodes'])[0])
sumz = 0
for syn in outsyn_list:
    z = (std_z(syn, avg(insyn_list), len(outsyn_list)))
    sumz += abs(z)
    print(">> \t%s, \t%s"%(syn,z))

print("AVG ST.Z: %s"%(sumz/len(outsyn_list)))
print("AVG SQ ST.Z: %s"%(avg_std_z(outsyn_list,avg(insyn_list))))

25.0
>> 	25.0, 	0.0
>> 	291.0, 	53.2
>> 	25.0, 	0.0
>> 	37.0, 	2.4
>> 	12.0, 	-2.6
AVG ST.Z: 11.64
AVG SQ ST.Z: 119.22164233057687


In [162]:
# RANK
# from data_modeling EXAMPLE 1 (Cosine, un-normalized), ['Attack on Titan'] input,
# ['Sword Art Online', 'Dragon Ball Z', 'Code Geass: Lelouch of the Rebellion R2', 
#  'Death Note', 'One Punch Man'] output
in_list = ['Attack on Titan']
insyn_list = list(gen_df[gen_df['title_english'].isin(in_list)]['rank'])
for syn in insyn_list:
    print(syn)
out_list = ['Sword Art Online', 'Dragon Ball Z', 'Code Geass: Lelouch of the Rebellion R2', 'Death Note', 'One Punch Man']
outsyn_list = []
for t in out_list:
    outsyn_list.append(list(gen_df[gen_df['title_english'] == t]['rank'])[0])
sumz = 0
for syn in outsyn_list:
    z = (std_z(syn, avg(insyn_list), len(outsyn_list)))
    sumz += abs(z)
    print(">> \t%s, \t%s"%(syn,z))

print("AVG ST.Z: %s"%(sumz/len(outsyn_list)))
print("AVG SQ ST.Z: %s"%(avg_std_z(outsyn_list,avg(insyn_list))))

116
>> 	1440, 	264.8
>> 	249, 	26.6
>> 	17, 	-19.8
>> 	54, 	-12.4
>> 	46, 	-14.0
AVG ST.Z: 67.52000000000001
AVG SQ ST.Z: 598.1989635564408


In [163]:
# MEMBERS
# from data_modeling EXAMPLE 1 (Cosine, un-normalized), ['Attack on Titan'] input,
# ['Sword Art Online', 'Dragon Ball Z', 'Code Geass: Lelouch of the Rebellion R2', 
#  'Death Note', 'One Punch Man'] output
in_list = ['Attack on Titan']
insyn_list = list(gen_df[gen_df['title_english'].isin(in_list)]['members'])
for syn in insyn_list:
    print(syn)
out_list = ['Sword Art Online', 'Dragon Ball Z', 'Code Geass: Lelouch of the Rebellion R2', 'Death Note', 'One Punch Man']
outsyn_list = []
for t in out_list:
    outsyn_list.append(list(gen_df[gen_df['title_english'] == t]['members'])[0])
sumz = 0
for syn in outsyn_list:
    z = (std_z(syn, avg(insyn_list), len(outsyn_list)))
    sumz += abs(z)
    print(">> \t%s, \t%s"%(syn,z))

print("AVG ST.Z: %s"%(sumz/len(outsyn_list)))
print("AVG SQ ST.Z: %s"%(avg_std_z(outsyn_list,avg(insyn_list))))

1500958
>> 	1442099, 	-11771.8
>> 	591990, 	-181793.6
>> 	869302, 	-126331.2
>> 	1610561, 	21920.6
>> 	1195384, 	-61114.8
AVG ST.Z: 80586.4
AVG SQ ST.Z: 516539.4072722816


In [164]:
# FAVORITES
# from data_modeling EXAMPLE 1 (Cosine, un-normalized), ['Attack on Titan'] input,
# ['Sword Art Online', 'Dragon Ball Z', 'Code Geass: Lelouch of the Rebellion R2', 
#  'Death Note', 'One Punch Man'] output
in_list = ['Attack on Titan']
insyn_list = list(gen_df[gen_df['title_english'].isin(in_list)]['favorites'])
for syn in insyn_list:
    print(syn)
out_list = ['Sword Art Online', 'Dragon Ball Z', 'Code Geass: Lelouch of the Rebellion R2', 'Death Note', 'One Punch Man']
outsyn_list = []
for t in out_list:
    outsyn_list.append(list(gen_df[gen_df['title_english'] == t]['favorites'])[0])
sumz = 0
for syn in outsyn_list:
    z = (std_z(syn, avg(insyn_list), len(outsyn_list)))
    sumz += abs(z)
    print(">> \t%s, \t%s"%(syn,z))

print("AVG ST.Z: %s"%(sumz/len(outsyn_list)))
print("AVG SQ ST.Z: %s"%(avg_std_z(outsyn_list,avg(insyn_list))))

70555
>> 	53268, 	-3457.4
>> 	23468, 	-9417.4
>> 	47637, 	-4583.6
>> 	96146, 	5118.2
>> 	35969, 	-6917.2
AVG ST.Z: 5898.76
AVG SQ ST.Z: 31280.509391632353


In [165]:
# from data_modeling EXAMPLE 1 (Cosine, normalized), ['Attack on Titan'] input,
# ['Attack on Titan Season 2', 'Fullmetal Alchemist: Brotherhood', 'Death Note',
# 'Code Geass: Lelouch of the Rebellion', 'Code Geass: Lelouch of the Rebellion R2'] output
in_list = ['Attack on Titan']
insyn_list = list(out_df[out_df['title_english'].isin(in_list)]['synopsis'])
for syn in insyn_list:
    print(syn[:300]+"...")
out_list = ['Attack on Titan Season 2', 'Fullmetal Alchemist: Brotherhood', 'Death Note',
            'Code Geass: Lelouch of the Rebellion', 'Code Geass: Lelouch of the Rebellion R2']
outsyn_list = []
for t in out_list:
    outsyn_list.append(list(out_df[out_df['title_english'] == t]['synopsis'])[0])
for syn in outsyn_list:
    print('>> \t%s'%(syn[:300]+"..."))

Centuries ago, mankind was slaughtered to near extinction by monstrous humanoid creatures called titans, forcing humans to hide in fear behind enormous concentric walls. What makes these giants truly terrifying is that their taste for human flesh is not born out of hunger but what appears to be out ...
>> 	For centuries, humanity has been hunted by giant, mysterious predators known as the Titans. Three mighty walls—Wall Maria, Rose, and Sheena—provided peace and protection for humanity for over a hundred years. That peace, however, was shattered when the Colossus Titan and Armored Titan appeared and d...
>> 	"In order for something to be obtained, something of equal value must be lost." Alchemy is bound by this Law of Equivalent Exchange—something the young brothers Edward and Alphonse Elric only realize after attempting human transmutation: the one forbidden act of alchemy. They pay a terrible price fo...
>> 	A shinigami, as a god of death, can kill any person—provided they see their 

In [166]:
# SCORED_BY
# from data_modeling EXAMPLE 1 (Cosine, un-normalized), ['Attack on Titan'] input,
# ['Sword Art Online', 'Dragon Ball Z', 'Code Geass: Lelouch of the Rebellion R2', 
#  'Death Note', 'One Punch Man'] output
in_list = ['Attack on Titan']
insyn_list = list(gen_df[gen_df['title_english'].isin(in_list)]['scored_by'])
for syn in insyn_list:
    print(syn)
out_list = ['Attack on Titan Season 2', 'Fullmetal Alchemist: Brotherhood', 'Death Note',
            'Code Geass: Lelouch of the Rebellion', 'Code Geass: Lelouch of the Rebellion R2']
outsyn_list = []
for t in out_list:
    outsyn_list.append(list(gen_df[gen_df['title_english'] == t]['scored_by'])[0])
sumz = 0
for syn in outsyn_list:
    z = (std_z(syn, avg(insyn_list), len(outsyn_list)))
    sumz += abs(z)
    print(">> \t%s, \t%s"%(syn,z))

print("AVG ST.Z: %s"%(sumz/len(outsyn_list)))
print("AVG SQ ST.Z: %s"%(avg_std_z(outsyn_list,avg(insyn_list))))

1038161
>> 	439965, 	-119639.2
>> 	826899, 	-42252.4
>> 	1107955, 	13958.8
>> 	681621, 	-71308.0
>> 	591272, 	-89377.8
AVG ST.Z: 67307.24
AVG SQ ST.Z: 383189.72831144626


In [167]:
# POPULARITY
# from data_modeling EXAMPLE 1 (Cosine, un-normalized), ['Attack on Titan'] input,
# ['Sword Art Online', 'Dragon Ball Z', 'Code Geass: Lelouch of the Rebellion R2', 
#  'Death Note', 'One Punch Man'] output
in_list = ['Attack on Titan']
insyn_list = list(gen_df[gen_df['title_english'].isin(in_list)]['popularity'])
for syn in insyn_list:
    print(syn)
out_list = ['Attack on Titan Season 2', 'Fullmetal Alchemist: Brotherhood', 'Death Note',
            'Code Geass: Lelouch of the Rebellion', 'Code Geass: Lelouch of the Rebellion R2']
outsyn_list = []
for t in out_list:
    outsyn_list.append(list(gen_df[gen_df['title_english'] == t]['popularity'])[0])
sumz = 0
for syn in outsyn_list:
    z = (std_z(syn, avg(insyn_list), len(outsyn_list)))
    sumz += abs(z)
    print(">> \t%s, \t%s"%(syn,z))

print("AVG ST.Z: %s"%(sumz/len(outsyn_list)))
print("AVG SQ ST.Z: %s"%(avg_std_z(outsyn_list,avg(insyn_list))))

2
>> 	41, 	7.8
>> 	4, 	0.4
>> 	1, 	-0.2
>> 	11, 	1.8
>> 	24, 	4.4
AVG ST.Z: 2.92
AVG SQ ST.Z: 20.449938875214272


In [168]:
# EPISODES
# from data_modeling EXAMPLE 1 (Cosine, un-normalized), ['Attack on Titan'] input,
# ['Sword Art Online', 'Dragon Ball Z', 'Code Geass: Lelouch of the Rebellion R2', 
#  'Death Note', 'One Punch Man'] output
in_list = ['Attack on Titan']
insyn_list = list(gen_df[gen_df['title_english'].isin(in_list)]['episodes'])
for syn in insyn_list:
    print(syn)
out_list = ['Attack on Titan Season 2', 'Fullmetal Alchemist: Brotherhood', 'Death Note',
            'Code Geass: Lelouch of the Rebellion', 'Code Geass: Lelouch of the Rebellion R2']
outsyn_list = []
for t in out_list:
    outsyn_list.append(list(gen_df[gen_df['title_english'] == t]['episodes'])[0])
sumz = 0
for syn in outsyn_list:
    z = (std_z(syn, avg(insyn_list), len(outsyn_list)))
    sumz += abs(z)
    print(">> \t%s, \t%s"%(syn,z))

print("AVG ST.Z: %s"%(sumz/len(outsyn_list)))
print("AVG SQ ST.Z: %s"%(avg_std_z(outsyn_list,avg(insyn_list))))

25.0
>> 	12.0, 	-2.6
>> 	64.0, 	7.8
>> 	37.0, 	2.4
>> 	25.0, 	0.0
>> 	25.0, 	0.0
AVG ST.Z: 2.56
AVG SQ ST.Z: 19.15202339179858


In [169]:
# RANK
# from data_modeling EXAMPLE 1 (Cosine, un-normalized), ['Attack on Titan'] input,
# ['Sword Art Online', 'Dragon Ball Z', 'Code Geass: Lelouch of the Rebellion R2', 
#  'Death Note', 'One Punch Man'] output
in_list = ['Attack on Titan']
insyn_list = list(gen_df[gen_df['title_english'].isin(in_list)]['rank'])
for syn in insyn_list:
    print(syn)
out_list = ['Attack on Titan Season 2', 'Fullmetal Alchemist: Brotherhood', 'Death Note',
            'Code Geass: Lelouch of the Rebellion', 'Code Geass: Lelouch of the Rebellion R2']
outsyn_list = []
for t in out_list:
    outsyn_list.append(list(gen_df[gen_df['title_english'] == t]['rank'])[0])
sumz = 0
for syn in outsyn_list:
    z = (std_z(syn, avg(insyn_list), len(outsyn_list)))
    sumz += abs(z)
    print(">> \t%s, \t%s"%(syn,z))

print("AVG ST.Z: %s"%(sumz/len(outsyn_list)))
print("AVG SQ ST.Z: %s"%(avg_std_z(outsyn_list,avg(insyn_list))))

116
>> 	143, 	5.4
>> 	1, 	-23.0
>> 	54, 	-12.4
>> 	32, 	-16.8
>> 	17, 	-19.8
AVG ST.Z: 15.479999999999999
AVG SQ ST.Z: 83.25262758616091


In [170]:
# MEMBERS
# from data_modeling EXAMPLE 1 (Cosine, un-normalized), ['Attack on Titan'] input,
# ['Sword Art Online', 'Dragon Ball Z', 'Code Geass: Lelouch of the Rebellion R2', 
#  'Death Note', 'One Punch Man'] output
in_list = ['Attack on Titan']
insyn_list = list(gen_df[gen_df['title_english'].isin(in_list)]['members'])
for syn in insyn_list:
    print(syn)
out_list = ['Attack on Titan Season 2', 'Fullmetal Alchemist: Brotherhood', 'Death Note',
            'Code Geass: Lelouch of the Rebellion', 'Code Geass: Lelouch of the Rebellion R2']
outsyn_list = []
for t in out_list:
    outsyn_list.append(list(gen_df[gen_df['title_english'] == t]['members'])[0])
sumz = 0
for syn in outsyn_list:
    z = (std_z(syn, avg(insyn_list), len(outsyn_list)))
    sumz += abs(z)
    print(">> \t%s, \t%s"%(syn,z))

print("AVG ST.Z: %s"%(sumz/len(outsyn_list)))
print("AVG SQ ST.Z: %s"%(avg_std_z(outsyn_list,avg(insyn_list))))

1500958
>> 	781129, 	-143965.8
>> 	1355349, 	-29121.8
>> 	1610561, 	21920.6
>> 	1082349, 	-83721.8
>> 	869302, 	-126331.2
AVG ST.Z: 81012.24
AVG SQ ST.Z: 474466.3075389021


In [171]:
# FAVORITES
# from data_modeling EXAMPLE 1 (Cosine, un-normalized), ['Attack on Titan'] input,
# ['Sword Art Online', 'Dragon Ball Z', 'Code Geass: Lelouch of the Rebellion R2', 
#  'Death Note', 'One Punch Man'] output
in_list = ['Attack on Titan']
insyn_list = list(gen_df[gen_df['title_english'].isin(in_list)]['favorites'])
for syn in insyn_list:
    print(syn)
out_list = ['Attack on Titan Season 2', 'Fullmetal Alchemist: Brotherhood', 'Death Note',
            'Code Geass: Lelouch of the Rebellion', 'Code Geass: Lelouch of the Rebellion R2']
outsyn_list = []
for t in out_list:
    outsyn_list.append(list(gen_df[gen_df['title_english'] == t]['favorites'])[0])
sumz = 0
for syn in outsyn_list:
    z = (std_z(syn, avg(insyn_list), len(outsyn_list)))
    sumz += abs(z)
    print(">> \t%s, \t%s"%(syn,z))

print("AVG ST.Z: %s"%(sumz/len(outsyn_list)))
print("AVG SQ ST.Z: %s"%(avg_std_z(outsyn_list,avg(insyn_list))))

70555
>> 	8670, 	-12377.0
>> 	120331, 	9955.2
>> 	96146, 	5118.2
>> 	68684, 	-374.2
>> 	47637, 	-4583.6
AVG ST.Z: 6481.640000000001
AVG SQ ST.Z: 38706.68688224297


In [172]:
# from data_modeling EXAMPLE 2 (Cosine, un-normalized)
# ['Attack on Titan', 'Attack on Titan: Since That Day', 'Attack on Titan: Crimson Bow and Arrow',
#  'Attack on Titan: Wings of Freedom', 'Attack on Titan Season 2', 'Attack on Titan: Junior High',
#  'Attack on Titan Season 3'] input,
# ['anohana: The Flower We Saw That Day', 'Puella Magi Madoka Magica the Movie: Rebellion',
#  'Kuroko\'s Basketball', 'Vampire Knight', 'Maid Sama!'] output
in_list = ['Attack on Titan', 'Attack on Titan: Since That Day', 'Attack on Titan: Crimson Bow and Arrow',
           'Attack on Titan: Wings of Freedom', 'Attack on Titan Season 2', 'Attack on Titan: Junior High',
           'Attack on Titan Season 3']
insyn_list = list(out_df[out_df['title_english'].isin(in_list)]['synopsis'])
for syn in insyn_list:
    print(syn[:300]+"...")
out_list = ['anohana: The Flower We Saw That Day', 'Puella Magi Madoka Magica the Movie: Rebellion', 
            'Kuroko\'s Basketball', 'Vampire Knight', 'Maid Sama!']
outsyn_list = []
for t in out_list:
    outsyn_list.append(list(out_df[out_df['title_english'] == t]['synopsis'])[0])
for syn in outsyn_list:
    print('>> \t%s'%(syn[:300]+"..."))

Centuries ago, mankind was slaughtered to near extinction by monstrous humanoid creatures called titans, forcing humans to hide in fear behind enormous concentric walls. What makes these giants truly terrifying is that their taste for human flesh is not born out of hunger but what appears to be out ...
Recap of the events in Shingeki no Kyojin that happened prior to episode 14....
Recap of episodes 1-13....
Recap of episodes 14-25....
For centuries, humanity has been hunted by giant, mysterious predators known as the Titans. Three mighty walls—Wall Maria, Rose, and Sheena—provided peace and protection for humanity for over a hundred years. That peace, however, was shattered when the Colossus Titan and Armored Titan appeared and d...
On his first day of junior high, Eren Yeager comes face-to-face with a titan—and has his lunch stolen! From that day on, he holds a grudge against titans for taking his favorite food from him, a cheeseburger, vowing to eliminate their kind once and for all.

In [173]:
# SCORED_BY
# from data_modeling EXAMPLE 1 (Cosine, un-normalized), ['Attack on Titan'] input,
# ['Sword Art Online', 'Dragon Ball Z', 'Code Geass: Lelouch of the Rebellion R2', 
#  'Death Note', 'One Punch Man'] output
in_list = ['Attack on Titan', 'Attack on Titan: Since That Day', 'Attack on Titan: Crimson Bow and Arrow',
           'Attack on Titan: Wings of Freedom', 'Attack on Titan Season 2', 'Attack on Titan: Junior High',
           'Attack on Titan Season 3']
insyn_list = []
for t in out_list:
    insyn_list.append(list(gen_df[gen_df['title_english'] == t]['scored_by'])[0])
print(insyn_list)
out_list = ['anohana: The Flower We Saw That Day', 'Puella Magi Madoka Magica the Movie: Rebellion', 
            'Kuroko\'s Basketball', 'Vampire Knight', 'Maid Sama!']
outsyn_list = []
for t in out_list:
    outsyn_list.append(list(gen_df[gen_df['title_english'] == t]['scored_by'])[0])
sumz = 0
for syn in outsyn_list:
    z = (std_z(syn, avg(insyn_list), len(outsyn_list)))
    sumz += abs(z)
    print(">> \t%s, \t%s"%(syn,z))

print("AVG ST.Z: %s"%(sumz/len(outsyn_list)))
print("AVG SQ ST.Z: %s"%(avg_std_z(outsyn_list,avg(insyn_list))))

[439345, 123238, 315061, 220644, 339932]
>> 	439345, 	30340.2
>> 	123238, 	-32881.2
>> 	315061, 	5483.4
>> 	220644, 	-13400.0
>> 	339932, 	10457.6
AVG ST.Z: 18512.48
AVG SQ ST.Z: 107719.31402492312


In [174]:
# POPULARITY
# from data_modeling EXAMPLE 1 (Cosine, un-normalized), ['Attack on Titan'] input,
# ['Sword Art Online', 'Dragon Ball Z', 'Code Geass: Lelouch of the Rebellion R2', 
#  'Death Note', 'One Punch Man'] output
in_list = ['Attack on Titan', 'Attack on Titan: Since That Day', 'Attack on Titan: Crimson Bow and Arrow',
           'Attack on Titan: Wings of Freedom', 'Attack on Titan Season 2', 'Attack on Titan: Junior High',
           'Attack on Titan Season 3']
insyn_list = []
for t in out_list:
    insyn_list.append(list(gen_df[gen_df['title_english'] == t]['popularity'])[0])
print(insyn_list)
out_list = ['anohana: The Flower We Saw That Day', 'Puella Magi Madoka Magica the Movie: Rebellion', 
            'Kuroko\'s Basketball', 'Vampire Knight', 'Maid Sama!']
outsyn_list = []
for t in out_list:
    outsyn_list.append(list(gen_df[gen_df['title_english'] == t]['popularity'])[0])
sumz = 0
for syn in outsyn_list:
    z = (std_z(syn, avg(insyn_list), len(outsyn_list)))
    sumz += abs(z)
    print(">> \t%s, \t%s"%(syn,z))

print("AVG ST.Z: %s"%(sumz/len(outsyn_list)))
print("AVG SQ ST.Z: %s"%(avg_std_z(outsyn_list,avg(insyn_list))))

[49, 416, 88, 173, 76]
>> 	49, 	-22.28
>> 	416, 	51.12
>> 	88, 	-14.48
>> 	173, 	2.5199999999999987
>> 	76, 	-16.880000000000003
AVG ST.Z: 21.456
AVG SQ ST.Z: 134.36160165761646


In [175]:
# EPISODES
# from data_modeling EXAMPLE 1 (Cosine, un-normalized), ['Attack on Titan'] input,
# ['Sword Art Online', 'Dragon Ball Z', 'Code Geass: Lelouch of the Rebellion R2', 
#  'Death Note', 'One Punch Man'] output
in_list = ['Attack on Titan', 'Attack on Titan: Since That Day', 'Attack on Titan: Crimson Bow and Arrow',
           'Attack on Titan: Wings of Freedom', 'Attack on Titan Season 2', 'Attack on Titan: Junior High',
           'Attack on Titan Season 3']
insyn_list = []
for t in out_list:
    insyn_list.append(list(gen_df[gen_df['title_english'] == t]['episodes'])[0])
print(insyn_list)
out_list = ['anohana: The Flower We Saw That Day', 'Puella Magi Madoka Magica the Movie: Rebellion', 
            'Kuroko\'s Basketball', 'Vampire Knight', 'Maid Sama!']
outsyn_list = []
for t in out_list:
    outsyn_list.append(list(gen_df[gen_df['title_english'] == t]['episodes'])[0])
sumz = 0
for syn in outsyn_list:
    z = (std_z(syn, avg(insyn_list), len(outsyn_list)))
    sumz += abs(z)
    print(">> \t%s, \t%s"%(syn,z))

print("AVG ST.Z: %s"%(sumz/len(outsyn_list)))
print("AVG SQ ST.Z: %s"%(avg_std_z(outsyn_list,avg(insyn_list))))

[11.0, 1.0, 25.0, 13.0, 26.0]
>> 	11.0, 	-0.8399999999999999
>> 	1.0, 	-2.84
>> 	25.0, 	1.9600000000000002
>> 	13.0, 	-0.43999999999999984
>> 	26.0, 	2.16
AVG ST.Z: 1.6479999999999997
AVG SQ ST.Z: 9.346657156438337


In [176]:
# RANK
# from data_modeling EXAMPLE 1 (Cosine, un-normalized), ['Attack on Titan'] input,
# ['Sword Art Online', 'Dragon Ball Z', 'Code Geass: Lelouch of the Rebellion R2', 
#  'Death Note', 'One Punch Man'] output
in_list = ['Attack on Titan', 'Attack on Titan: Since That Day', 'Attack on Titan: Crimson Bow and Arrow',
           'Attack on Titan: Wings of Freedom', 'Attack on Titan Season 2', 'Attack on Titan: Junior High',
           'Attack on Titan Season 3']
insyn_list = []
for t in out_list:
    insyn_list.append(list(gen_df[gen_df['title_english'] == t]['rank'])[0])
print(insyn_list)
out_list = ['anohana: The Flower We Saw That Day', 'Puella Magi Madoka Magica the Movie: Rebellion', 
            'Kuroko\'s Basketball', 'Vampire Knight', 'Maid Sama!']
outsyn_list = []
for t in out_list:
    outsyn_list.append(list(gen_df[gen_df['title_english'] == t]['rank'])[0])
sumz = 0
for syn in outsyn_list:
    z = (std_z(syn, avg(insyn_list), len(outsyn_list)))
    sumz += abs(z)
    print(">> \t%s, \t%s"%(syn,z))

print("AVG ST.Z: %s"%(sumz/len(outsyn_list)))
print("AVG SQ ST.Z: %s"%(avg_std_z(outsyn_list,avg(insyn_list))))

[93, 131, 212, 2602, 336]
>> 	93, 	-116.35999999999999
>> 	131, 	-108.75999999999999
>> 	212, 	-92.55999999999999
>> 	2602, 	385.44
>> 	336, 	-67.75999999999999
AVG ST.Z: 154.176
AVG SQ ST.Z: 967.1896194645599


In [177]:
# MEMBERS
# from data_modeling EXAMPLE 1 (Cosine, un-normalized), ['Attack on Titan'] input,
# ['Sword Art Online', 'Dragon Ball Z', 'Code Geass: Lelouch of the Rebellion R2', 
#  'Death Note', 'One Punch Man'] output
in_list = ['Attack on Titan', 'Attack on Titan: Since That Day', 'Attack on Titan: Crimson Bow and Arrow',
           'Attack on Titan: Wings of Freedom', 'Attack on Titan Season 2', 'Attack on Titan: Junior High',
           'Attack on Titan Season 3']
insyn_list = []
for t in out_list:
    insyn_list.append(list(gen_df[gen_df['title_english'] == t]['members'])[0])
print(insyn_list)
out_list = ['anohana: The Flower We Saw That Day', 'Puella Magi Madoka Magica the Movie: Rebellion', 
            'Kuroko\'s Basketball', 'Vampire Knight', 'Maid Sama!']
outsyn_list = []
for t in out_list:
    outsyn_list.append(list(gen_df[gen_df['title_english'] == t]['members'])[0])
sumz = 0
for syn in outsyn_list:
    z = (std_z(syn, avg(insyn_list), len(outsyn_list)))
    sumz += abs(z)
    print(">> \t%s, \t%s"%(syn,z))

print("AVG ST.Z: %s"%(sumz/len(outsyn_list)))
print("AVG SQ ST.Z: %s"%(avg_std_z(outsyn_list,avg(insyn_list))))

[734942, 208482, 534177, 371097, 579513]
>> 	734942, 	49859.96
>> 	208482, 	-55432.04
>> 	534177, 	9706.959999999997
>> 	371097, 	-22909.04
>> 	579513, 	18774.159999999996
AVG ST.Z: 31336.432
AVG SQ ST.Z: 180696.49885418368


In [178]:
# FAVORITES
# from data_modeling EXAMPLE 1 (Cosine, un-normalized), ['Attack on Titan'] input,
# ['Sword Art Online', 'Dragon Ball Z', 'Code Geass: Lelouch of the Rebellion R2', 
#  'Death Note', 'One Punch Man'] output
in_list = ['Attack on Titan', 'Attack on Titan: Since That Day', 'Attack on Titan: Crimson Bow and Arrow',
           'Attack on Titan: Wings of Freedom', 'Attack on Titan Season 2', 'Attack on Titan: Junior High',
           'Attack on Titan Season 3']
insyn_list = []
for t in out_list:
    insyn_list.append(list(gen_df[gen_df['title_english'] == t]['favorites'])[0])
print(insyn_list)
out_list = ['anohana: The Flower We Saw That Day', 'Puella Magi Madoka Magica the Movie: Rebellion', 
            'Kuroko\'s Basketball', 'Vampire Knight', 'Maid Sama!']
outsyn_list = []
for t in out_list:
    outsyn_list.append(list(gen_df[gen_df['title_english'] == t]['favorites'])[0])
sumz = 0
for syn in outsyn_list:
    z = (std_z(syn, avg(insyn_list), len(outsyn_list)))
    sumz += abs(z)
    print(">> \t%s, \t%s"%(syn,z))

print("AVG ST.Z: %s"%(sumz/len(outsyn_list)))
print("AVG SQ ST.Z: %s"%(avg_std_z(outsyn_list,avg(insyn_list))))

[22969, 5275, 16005, 8111, 15410]
>> 	22969, 	1883.0
>> 	5275, 	-1655.8
>> 	16005, 	490.2
>> 	8111, 	-1088.6
>> 	15410, 	371.2
AVG ST.Z: 1097.76
AVG SQ ST.Z: 6265.180795475897


In [179]:
# from data_modeling EXAMPLE 2 (Cosine, normalized)
# ['Attack on Titan', 'Attack on Titan: Since That Day', 'Attack on Titan: Crimson Bow and Arrow',
#  'Attack on Titan: Wings of Freedom', 'Attack on Titan Season 2', 'Attack on Titan: Junior High',
#  'Attack on Titan Season 3'] input,
# ['Gun Samurai Recap', 'Marches Comes in Like a Lion: Special Omnibus Episode',
#  'Berserk: Recollections of the Witch', 'So, I Can\'t Play H!',
#  'Tsukigakirei: First Half - The Road So Far'] output
in_list = ['Attack on Titan', 'Attack on Titan: Since That Day', 'Attack on Titan: Crimson Bow and Arrow',
           'Attack on Titan: Wings of Freedom', 'Attack on Titan Season 2', 'Attack on Titan: Junior High',
           'Attack on Titan Season 3']
insyn_list = list(out_df[out_df['title_english'].isin(in_list)]['synopsis'])
for syn in insyn_list:
    print(syn[:300]+"...")
out_list = ['Gun Samurai Recap', 'Marches Comes in Like a Lion: Special Omnibus Episode',
            'Berserk: Recollections of the Witch', 'So, I Can\'t Play H!',
            'Tsukigakirei: First Half - The Road So Far']
outsyn_list = []
for t in out_list:
    outsyn_list.append(list(out_df[out_df['title_english'] == t]['synopsis'])[0])
for syn in outsyn_list:
    print('>> \t%s'%(syn[:300]+"..."))

Centuries ago, mankind was slaughtered to near extinction by monstrous humanoid creatures called titans, forcing humans to hide in fear behind enormous concentric walls. What makes these giants truly terrifying is that their taste for human flesh is not born out of hunger but what appears to be out ...
Recap of the events in Shingeki no Kyojin that happened prior to episode 14....
Recap of episodes 1-13....
Recap of episodes 14-25....
For centuries, humanity has been hunted by giant, mysterious predators known as the Titans. Three mighty walls—Wall Maria, Rose, and Sheena—provided peace and protection for humanity for over a hundred years. That peace, however, was shattered when the Colossus Titan and Armored Titan appeared and d...
On his first day of junior high, Eren Yeager comes face-to-face with a titan—and has his lunch stolen! From that day on, he holds a grudge against titans for taking his favorite food from him, a cheeseburger, vowing to eliminate their kind once and for all.

In [180]:
# SCORED_BY
# from data_modeling EXAMPLE 1 (Cosine, un-normalized), ['Attack on Titan'] input,
# ['Sword Art Online', 'Dragon Ball Z', 'Code Geass: Lelouch of the Rebellion R2', 
#  'Death Note', 'One Punch Man'] output
in_list = ['Attack on Titan', 'Attack on Titan: Since That Day', 'Attack on Titan: Crimson Bow and Arrow',
           'Attack on Titan: Wings of Freedom', 'Attack on Titan Season 2', 'Attack on Titan: Junior High',
           'Attack on Titan Season 3']
insyn_list = []
for t in out_list:
    insyn_list.append(list(gen_df[gen_df['title_english'] == t]['scored_by'])[0])
print(insyn_list)
out_list = ['Gun Samurai Recap', 'Marches Comes in Like a Lion: Special Omnibus Episode',
            'Berserk: Recollections of the Witch', 'So, I Can\'t Play H!',
            'Tsukigakirei: First Half - The Road So Far']
outsyn_list = []
for t in out_list:
    outsyn_list.append(list(gen_df[gen_df['title_english'] == t]['scored_by'])[0])
sumz = 0
for syn in outsyn_list:
    z = (std_z(syn, avg(insyn_list), len(outsyn_list)))
    sumz += abs(z)
    print(">> \t%s, \t%s"%(syn,z))

print("AVG ST.Z: %s"%(sumz/len(outsyn_list)))
print("AVG SQ ST.Z: %s"%(avg_std_z(outsyn_list,avg(insyn_list))))

[45, 4589, 4083, 133619, 5019]
>> 	45, 	-5885.2
>> 	4589, 	-4976.4
>> 	4083, 	-5077.6
>> 	133619, 	20829.6
>> 	5019, 	-4890.4
AVG ST.Z: 8331.84
AVG SQ ST.Z: 52104.241961667576


In [181]:
# POPULARITY
# from data_modeling EXAMPLE 1 (Cosine, un-normalized), ['Attack on Titan'] input,
# ['Sword Art Online', 'Dragon Ball Z', 'Code Geass: Lelouch of the Rebellion R2', 
#  'Death Note', 'One Punch Man'] output
in_list = ['Attack on Titan', 'Attack on Titan: Since That Day', 'Attack on Titan: Crimson Bow and Arrow',
           'Attack on Titan: Wings of Freedom', 'Attack on Titan Season 2', 'Attack on Titan: Junior High',
           'Attack on Titan Season 3']
insyn_list = []
for t in out_list:
    insyn_list.append(list(gen_df[gen_df['title_english'] == t]['popularity'])[0])
print(insyn_list)
out_list = ['Gun Samurai Recap', 'Marches Comes in Like a Lion: Special Omnibus Episode',
            'Berserk: Recollections of the Witch', 'So, I Can\'t Play H!',
            'Tsukigakirei: First Half - The Road So Far']
outsyn_list = []
for t in out_list:
    outsyn_list.append(list(gen_df[gen_df['title_english'] == t]['popularity'])[0])
sumz = 0
for syn in outsyn_list:
    z = (std_z(syn, avg(insyn_list), len(outsyn_list)))
    sumz += abs(z)
    print(">> \t%s, \t%s"%(syn,z))

print("AVG ST.Z: %s"%(sumz/len(outsyn_list)))
print("AVG SQ ST.Z: %s"%(avg_std_z(outsyn_list,avg(insyn_list))))

[11050, 4185, 4249, 320, 4118]
>> 	11050, 	1253.1200000000001
>> 	4185, 	-119.87999999999992
>> 	4249, 	-107.07999999999993
>> 	320, 	-892.8799999999999
>> 	4118, 	-133.27999999999992
AVG ST.Z: 501.24799999999993
AVG SQ ST.Z: 3472.135746194264


In [182]:
# EPISODES
# from data_modeling EXAMPLE 1 (Cosine, un-normalized), ['Attack on Titan'] input,
# ['Sword Art Online', 'Dragon Ball Z', 'Code Geass: Lelouch of the Rebellion R2', 
#  'Death Note', 'One Punch Man'] output
in_list = ['Attack on Titan', 'Attack on Titan: Since That Day', 'Attack on Titan: Crimson Bow and Arrow',
           'Attack on Titan: Wings of Freedom', 'Attack on Titan Season 2', 'Attack on Titan: Junior High',
           'Attack on Titan Season 3']
insyn_list = []
for t in out_list:
    insyn_list.append(list(gen_df[gen_df['title_english'] == t]['episodes'])[0])
print(insyn_list)
out_list = ['Gun Samurai Recap', 'Marches Comes in Like a Lion: Special Omnibus Episode',
            'Berserk: Recollections of the Witch', 'So, I Can\'t Play H!',
            'Tsukigakirei: First Half - The Road So Far']
outsyn_list = []
for t in out_list:
    outsyn_list.append(list(gen_df[gen_df['title_english'] == t]['episodes'])[0])
sumz = 0
for syn in outsyn_list:
    z = (std_z(syn, avg(insyn_list), len(outsyn_list)))
    sumz += abs(z)
    print(">> \t%s, \t%s"%(syn,z))

print("AVG ST.Z: %s"%(sumz/len(outsyn_list)))
print("AVG SQ ST.Z: %s"%(avg_std_z(outsyn_list,avg(insyn_list))))

[1.0, 1.0, 1.0, 12.0, 1.0]
>> 	1.0, 	-0.44000000000000006
>> 	1.0, 	-0.44000000000000006
>> 	1.0, 	-0.44000000000000006
>> 	12.0, 	1.7600000000000002
>> 	1.0, 	-0.44000000000000006
AVG ST.Z: 0.7040000000000001
AVG SQ ST.Z: 4.4


In [183]:
# RANK
# from data_modeling EXAMPLE 1 (Cosine, un-normalized), ['Attack on Titan'] input,
# ['Sword Art Online', 'Dragon Ball Z', 'Code Geass: Lelouch of the Rebellion R2', 
#  'Death Note', 'One Punch Man'] output
in_list = ['Attack on Titan', 'Attack on Titan: Since That Day', 'Attack on Titan: Crimson Bow and Arrow',
           'Attack on Titan: Wings of Freedom', 'Attack on Titan Season 2', 'Attack on Titan: Junior High',
           'Attack on Titan Season 3']
insyn_list = []
for t in out_list:
    insyn_list.append(list(gen_df[gen_df['title_english'] == t]['rank'])[0])
print(insyn_list)
out_list = ['Gun Samurai Recap', 'Marches Comes in Like a Lion: Special Omnibus Episode',
            'Berserk: Recollections of the Witch', 'So, I Can\'t Play H!',
            'Tsukigakirei: First Half - The Road So Far']
outsyn_list = []
for t in out_list:
    outsyn_list.append(list(gen_df[gen_df['title_english'] == t]['rank'])[0])
sumz = 0
for syn in outsyn_list:
    z = (std_z(syn, avg(insyn_list), len(outsyn_list)))
    sumz += abs(z)
    print(">> \t%s, \t%s"%(syn,z))

print("AVG ST.Z: %s"%(sumz/len(outsyn_list)))
print("AVG SQ ST.Z: %s"%(avg_std_z(outsyn_list,avg(insyn_list))))

[11051, 4747, 8229, 4241, 5603]
>> 	11051, 	855.36
>> 	4747, 	-405.43999999999994
>> 	8229, 	290.96000000000004
>> 	4241, 	-506.64
>> 	5603, 	-234.23999999999995
AVG ST.Z: 458.52799999999996
AVG SQ ST.Z: 2541.881696696367


In [184]:
# MEMBERS
# from data_modeling EXAMPLE 1 (Cosine, un-normalized), ['Attack on Titan'] input,
# ['Sword Art Online', 'Dragon Ball Z', 'Code Geass: Lelouch of the Rebellion R2', 
#  'Death Note', 'One Punch Man'] output
in_list = ['Attack on Titan', 'Attack on Titan: Since That Day', 'Attack on Titan: Crimson Bow and Arrow',
           'Attack on Titan: Wings of Freedom', 'Attack on Titan Season 2', 'Attack on Titan: Junior High',
           'Attack on Titan Season 3']
insyn_list = []
for t in out_list:
    insyn_list.append(list(gen_df[gen_df['title_english'] == t]['members'])[0])
print(insyn_list)
out_list = ['Gun Samurai Recap', 'Marches Comes in Like a Lion: Special Omnibus Episode',
            'Berserk: Recollections of the Witch', 'So, I Can\'t Play H!',
            'Tsukigakirei: First Half - The Road So Far']
outsyn_list = []
for t in out_list:
    outsyn_list.append(list(gen_df[gen_df['title_english'] == t]['members'])[0])
sumz = 0
for syn in outsyn_list:
    z = (std_z(syn, avg(insyn_list), len(outsyn_list)))
    sumz += abs(z)
    print(">> \t%s, \t%s"%(syn,z))

print("AVG ST.Z: %s"%(sumz/len(outsyn_list)))
print("AVG SQ ST.Z: %s"%(avg_std_z(outsyn_list,avg(insyn_list))))

[328, 8811, 8518, 255584, 9212]
>> 	328, 	-11232.52
>> 	8811, 	-9535.92
>> 	8518, 	-9594.52
>> 	255584, 	39818.68
>> 	9212, 	-9455.72
AVG ST.Z: 15927.472
AVG SQ ST.Z: 99601.60660069695


In [185]:
# FAVORITES
# from data_modeling EXAMPLE 1 (Cosine, un-normalized), ['Attack on Titan'] input,
# ['Sword Art Online', 'Dragon Ball Z', 'Code Geass: Lelouch of the Rebellion R2', 
#  'Death Note', 'One Punch Man'] output
in_list = ['Attack on Titan', 'Attack on Titan: Since That Day', 'Attack on Titan: Crimson Bow and Arrow',
           'Attack on Titan: Wings of Freedom', 'Attack on Titan Season 2', 'Attack on Titan: Junior High',
           'Attack on Titan Season 3']
insyn_list = []
for t in out_list:
    insyn_list.append(list(gen_df[gen_df['title_english'] == t]['favorites'])[0])
print(insyn_list)
out_list = ['Gun Samurai Recap', 'Marches Comes in Like a Lion: Special Omnibus Episode',
            'Berserk: Recollections of the Witch', 'So, I Can\'t Play H!',
            'Tsukigakirei: First Half - The Road So Far']
outsyn_list = []
for t in out_list:
    outsyn_list.append(list(gen_df[gen_df['title_english'] == t]['favorites'])[0])
sumz = 0
for syn in outsyn_list:
    z = (std_z(syn, avg(insyn_list), len(outsyn_list)))
    sumz += abs(z)
    print(">> \t%s, \t%s"%(syn,z))

print("AVG ST.Z: %s"%(sumz/len(outsyn_list)))
print("AVG SQ ST.Z: %s"%(avg_std_z(outsyn_list,avg(insyn_list))))

[0, 8, 13, 701, 10]
>> 	0, 	-29.28
>> 	8, 	-27.68
>> 	13, 	-26.68
>> 	701, 	110.92
>> 	10, 	-27.28
AVG ST.Z: 44.368
AVG SQ ST.Z: 277.3334455127978


In [186]:
# from data_modeling EXAMPLE 2 (Euclidean, normalized)
# ['Attack on Titan', 'Attack on Titan: Since That Day', 'Attack on Titan: Crimson Bow and Arrow',
#  'Attack on Titan: Wings of Freedom', 'Attack on Titan Season 2', 'Attack on Titan: Junior High',
#  'Attack on Titan Season 3'] input,
# ['Gun Samurai Recap', 'Marches Comes in Like a Lion: Special Omnibus Episode',
#  'Berserk: Recollections of the Witch', 'So, I Can\'t Play H!',
#  'Tsukigakirei: First Half - The Road So Far'] output
in_list = ['Attack on Titan', 'Attack on Titan: Since That Day', 'Attack on Titan: Crimson Bow and Arrow',
           'Attack on Titan: Wings of Freedom', 'Attack on Titan Season 2', 'Attack on Titan: Junior High',
           'Attack on Titan Season 3']
insyn_list = list(out_df[out_df['title_english'].isin(in_list)]['synopsis'])
for syn in insyn_list:
    print(syn[:300]+"...")
out_list = ['Marches Comes in Like a Lion: Special Omnibus Episode', 
            'Persona 4 the Animation: The Factor of Hope', 
            'Fullmetal Alchemist: Premium OVA Collection', 'Shiki Specials',
            'Robot Girls Z']
outsyn_list = []
for t in out_list:
    outsyn_list.append(list(out_df[out_df['title_english'] == t]['synopsis'])[0])
for syn in outsyn_list:
    print('>> \t%s'%(syn[:300]+"..."))

Centuries ago, mankind was slaughtered to near extinction by monstrous humanoid creatures called titans, forcing humans to hide in fear behind enormous concentric walls. What makes these giants truly terrifying is that their taste for human flesh is not born out of hunger but what appears to be out ...
Recap of the events in Shingeki no Kyojin that happened prior to episode 14....
Recap of episodes 1-13....
Recap of episodes 14-25....
For centuries, humanity has been hunted by giant, mysterious predators known as the Titans. Three mighty walls—Wall Maria, Rose, and Sheena—provided peace and protection for humanity for over a hundred years. That peace, however, was shattered when the Colossus Titan and Armored Titan appeared and d...
On his first day of junior high, Eren Yeager comes face-to-face with a titan—and has his lunch stolen! From that day on, he holds a grudge against titans for taking his favorite food from him, a cheeseburger, vowing to eliminate their kind once and for all.

In [187]:
# SCORED_BY
# from data_modeling EXAMPLE 1 (Cosine, un-normalized), ['Attack on Titan'] input,
# ['Sword Art Online', 'Dragon Ball Z', 'Code Geass: Lelouch of the Rebellion R2', 
#  'Death Note', 'One Punch Man'] output
in_list = ['Attack on Titan', 'Attack on Titan: Since That Day', 'Attack on Titan: Crimson Bow and Arrow',
           'Attack on Titan: Wings of Freedom', 'Attack on Titan Season 2', 'Attack on Titan: Junior High',
           'Attack on Titan Season 3']
insyn_list = []
for t in out_list:
    insyn_list.append(list(gen_df[gen_df['title_english'] == t]['scored_by'])[0])
print(insyn_list)
out_list = ['Marches Comes in Like a Lion: Special Omnibus Episode', 
            'Persona 4 the Animation: The Factor of Hope', 
            'Fullmetal Alchemist: Premium OVA Collection', 'Shiki Specials',
            'Robot Girls Z']
outsyn_list = []
for t in out_list:
    outsyn_list.append(list(gen_df[gen_df['title_english'] == t]['scored_by'])[0])
sumz = 0
for syn in outsyn_list:
    z = (std_z(syn, avg(insyn_list), len(outsyn_list)))
    sumz += abs(z)
    print(">> \t%s, \t%s"%(syn,z))

print("AVG ST.Z: %s"%(sumz/len(outsyn_list)))
print("AVG SQ ST.Z: %s"%(avg_std_z(outsyn_list,avg(insyn_list))))

[4589, 3261, 27630, 28578, 6135]
>> 	4589, 	-1889.92
>> 	3261, 	-2155.52
>> 	27630, 	2718.2799999999997
>> 	28578, 	2907.88
>> 	6135, 	-1580.72
AVG ST.Z: 2250.4639999999995
AVG SQ ST.Z: 11524.22493012003


In [188]:
# POPULARITY
# from data_modeling EXAMPLE 1 (Cosine, un-normalized), ['Attack on Titan'] input,
# ['Sword Art Online', 'Dragon Ball Z', 'Code Geass: Lelouch of the Rebellion R2', 
#  'Death Note', 'One Punch Man'] output
in_list = ['Attack on Titan', 'Attack on Titan: Since That Day', 'Attack on Titan: Crimson Bow and Arrow',
           'Attack on Titan: Wings of Freedom', 'Attack on Titan Season 2', 'Attack on Titan: Junior High',
           'Attack on Titan Season 3']
insyn_list = []
for t in out_list:
    insyn_list.append(list(gen_df[gen_df['title_english'] == t]['popularity'])[0])
print(insyn_list)
out_list = ['Marches Comes in Like a Lion: Special Omnibus Episode', 
            'Persona 4 the Animation: The Factor of Hope', 
            'Fullmetal Alchemist: Premium OVA Collection', 'Shiki Specials',
            'Robot Girls Z']
outsyn_list = []
for t in out_list:
    outsyn_list.append(list(gen_df[gen_df['title_english'] == t]['popularity'])[0])
sumz = 0
for syn in outsyn_list:
    z = (std_z(syn, avg(insyn_list), len(outsyn_list)))
    sumz += abs(z)
    print(">> \t%s, \t%s"%(syn,z))

print("AVG ST.Z: %s"%(sumz/len(outsyn_list)))
print("AVG SQ ST.Z: %s"%(avg_std_z(outsyn_list,avg(insyn_list))))

[4185, 3487, 1605, 1701, 3234]
>> 	4185, 	268.52
>> 	3487, 	128.92
>> 	1605, 	-247.48000000000002
>> 	1701, 	-228.28000000000003
>> 	3234, 	78.31999999999998
AVG ST.Z: 190.304
AVG SQ ST.Z: 1020.3320243920602


In [189]:
# EPISODES
# from data_modeling EXAMPLE 1 (Cosine, un-normalized), ['Attack on Titan'] input,
# ['Sword Art Online', 'Dragon Ball Z', 'Code Geass: Lelouch of the Rebellion R2', 
#  'Death Note', 'One Punch Man'] output
in_list = ['Attack on Titan', 'Attack on Titan: Since That Day', 'Attack on Titan: Crimson Bow and Arrow',
           'Attack on Titan: Wings of Freedom', 'Attack on Titan Season 2', 'Attack on Titan: Junior High',
           'Attack on Titan Season 3']
insyn_list = []
for t in out_list:
    insyn_list.append(list(gen_df[gen_df['title_english'] == t]['episodes'])[0])
print(insyn_list)
out_list = ['Marches Comes in Like a Lion: Special Omnibus Episode', 
            'Persona 4 the Animation: The Factor of Hope', 
            'Fullmetal Alchemist: Premium OVA Collection', 'Shiki Specials',
            'Robot Girls Z']
outsyn_list = []
for t in out_list:
    outsyn_list.append(list(gen_df[gen_df['title_english'] == t]['episodes'])[0])
sumz = 0
for syn in outsyn_list:
    z = (std_z(syn, avg(insyn_list), len(outsyn_list)))
    sumz += abs(z)
    print(">> \t%s, \t%s"%(syn,z))

print("AVG ST.Z: %s"%(sumz/len(outsyn_list)))
print("AVG SQ ST.Z: %s"%(avg_std_z(outsyn_list,avg(insyn_list))))

[1.0, 1.0, 3.0, 2.0, 3.0]
>> 	1.0, 	-0.2
>> 	1.0, 	-0.2
>> 	3.0, 	0.2
>> 	2.0, 	0.0
>> 	3.0, 	0.2
AVG ST.Z: 0.16
AVG SQ ST.Z: 0.8944271909999159


In [190]:
# RANK
# from data_modeling EXAMPLE 1 (Cosine, un-normalized), ['Attack on Titan'] input,
# ['Sword Art Online', 'Dragon Ball Z', 'Code Geass: Lelouch of the Rebellion R2', 
#  'Death Note', 'One Punch Man'] output
in_list = ['Attack on Titan', 'Attack on Titan: Since That Day', 'Attack on Titan: Crimson Bow and Arrow',
           'Attack on Titan: Wings of Freedom', 'Attack on Titan Season 2', 'Attack on Titan: Junior High',
           'Attack on Titan Season 3']
insyn_list = []
for t in out_list:
    insyn_list.append(list(gen_df[gen_df['title_english'] == t]['rank'])[0])
print(insyn_list)
out_list = ['Marches Comes in Like a Lion: Special Omnibus Episode', 
            'Persona 4 the Animation: The Factor of Hope', 
            'Fullmetal Alchemist: Premium OVA Collection', 'Shiki Specials',
            'Robot Girls Z']
outsyn_list = []
for t in out_list:
    outsyn_list.append(list(gen_df[gen_df['title_english'] == t]['rank'])[0])
sumz = 0
for syn in outsyn_list:
    z = (std_z(syn, avg(insyn_list), len(outsyn_list)))
    sumz += abs(z)
    print(">> \t%s, \t%s"%(syn,z))

print("AVG ST.Z: %s"%(sumz/len(outsyn_list)))
print("AVG SQ ST.Z: %s"%(avg_std_z(outsyn_list,avg(insyn_list))))

[4747, 1879, 2000, 749, 5800]
>> 	4747, 	342.4
>> 	1879, 	-231.2
>> 	2000, 	-207.0
>> 	749, 	-457.2
>> 	5800, 	553.0
AVG ST.Z: 358.15999999999997
AVG SQ ST.Z: 1908.3776355847394


In [191]:
# MEMBERS
# from data_modeling EXAMPLE 1 (Cosine, un-normalized), ['Attack on Titan'] input,
# ['Sword Art Online', 'Dragon Ball Z', 'Code Geass: Lelouch of the Rebellion R2', 
#  'Death Note', 'One Punch Man'] output
in_list = ['Attack on Titan', 'Attack on Titan: Since That Day', 'Attack on Titan: Crimson Bow and Arrow',
           'Attack on Titan: Wings of Freedom', 'Attack on Titan Season 2', 'Attack on Titan: Junior High',
           'Attack on Titan Season 3']
insyn_list = []
for t in out_list:
    insyn_list.append(list(gen_df[gen_df['title_english'] == t]['members'])[0])
print(insyn_list)
out_list = ['Marches Comes in Like a Lion: Special Omnibus Episode', 
            'Persona 4 the Animation: The Factor of Hope', 
            'Fullmetal Alchemist: Premium OVA Collection', 'Shiki Specials',
            'Robot Girls Z']
outsyn_list = []
for t in out_list:
    outsyn_list.append(list(gen_df[gen_df['title_english'] == t]['members'])[0])
sumz = 0
for syn in outsyn_list:
    z = (std_z(syn, avg(insyn_list), len(outsyn_list)))
    sumz += abs(z)
    print(">> \t%s, \t%s"%(syn,z))

print("AVG ST.Z: %s"%(sumz/len(outsyn_list)))
print("AVG SQ ST.Z: %s"%(avg_std_z(outsyn_list,avg(insyn_list))))

[8811, 13700, 52727, 48923, 16045]
>> 	8811, 	-3846.04
>> 	13700, 	-2868.2400000000002
>> 	52727, 	4937.16
>> 	48923, 	4176.36
>> 	16045, 	-2399.2400000000002
AVG ST.Z: 3645.4080000000004
AVG SQ ST.Z: 18787.324007425857


In [192]:
# FAVORITES
# from data_modeling EXAMPLE 1 (Cosine, un-normalized), ['Attack on Titan'] input,
# ['Sword Art Online', 'Dragon Ball Z', 'Code Geass: Lelouch of the Rebellion R2', 
#  'Death Note', 'One Punch Man'] output
in_list = ['Attack on Titan', 'Attack on Titan: Since That Day', 'Attack on Titan: Crimson Bow and Arrow',
           'Attack on Titan: Wings of Freedom', 'Attack on Titan Season 2', 'Attack on Titan: Junior High',
           'Attack on Titan Season 3']
insyn_list = []
for t in out_list:
    insyn_list.append(list(gen_df[gen_df['title_english'] == t]['favorites'])[0])
print(insyn_list)
out_list = ['Marches Comes in Like a Lion: Special Omnibus Episode', 
            'Persona 4 the Animation: The Factor of Hope', 
            'Fullmetal Alchemist: Premium OVA Collection', 'Shiki Specials',
            'Robot Girls Z']
outsyn_list = []
for t in out_list:
    outsyn_list.append(list(gen_df[gen_df['title_english'] == t]['favorites'])[0])
sumz = 0
for syn in outsyn_list:
    z = (std_z(syn, avg(insyn_list), len(outsyn_list)))
    sumz += abs(z)
    print(">> \t%s, \t%s"%(syn,z))

print("AVG ST.Z: %s"%(sumz/len(outsyn_list)))
print("AVG SQ ST.Z: %s"%(avg_std_z(outsyn_list,avg(insyn_list))))

[8, 15, 101, 76, 14]
>> 	8, 	-6.959999999999999
>> 	15, 	-5.56
>> 	101, 	11.64
>> 	76, 	6.640000000000001
>> 	14, 	-5.76
AVG ST.Z: 7.312
AVG SQ ST.Z: 38.2172735814579


In [193]:
# from data_modeling EXAMPLE 2 (Euclidean, un-normalized)
# ['Attack on Titan', 'Attack on Titan: Since That Day', 'Attack on Titan: Crimson Bow and Arrow',
#  'Attack on Titan: Wings of Freedom', 'Attack on Titan Season 2', 'Attack on Titan: Junior High',
#  'Attack on Titan Season 3'] input,
# ['Gun Samurai Recap', 'Marches Comes in Like a Lion: Special Omnibus Episode',
#  'Berserk: Recollections of the Witch', 'So, I Can\'t Play H!',
#  'Tsukigakirei: First Half - The Road So Far'] output
in_list = ['Attack on Titan', 'Attack on Titan: Since That Day', 'Attack on Titan: Crimson Bow and Arrow',
           'Attack on Titan: Wings of Freedom', 'Attack on Titan Season 2', 'Attack on Titan: Junior High',
           'Attack on Titan Season 3']
insyn_list = list(out_df[out_df['title_english'].isin(in_list)]['synopsis'])
for syn in insyn_list:
    print(syn[:300]+"...")
out_list = ['Miss Kobayashi\'s Dragon Maid', 'Rosario + Vampire', 
            'My Teen Romantic Comedy SNAFU TOO!', 'GATE',
            'JoJo\'s Bizarre Adventure']
outsyn_list = []
for t in out_list:
    outsyn_list.append(list(out_df[out_df['title_english'] == t]['synopsis'])[0])
for syn in outsyn_list:
    print('>> \t%s'%(syn[:300]+"..."))

Centuries ago, mankind was slaughtered to near extinction by monstrous humanoid creatures called titans, forcing humans to hide in fear behind enormous concentric walls. What makes these giants truly terrifying is that their taste for human flesh is not born out of hunger but what appears to be out ...
Recap of the events in Shingeki no Kyojin that happened prior to episode 14....
Recap of episodes 1-13....
Recap of episodes 14-25....
For centuries, humanity has been hunted by giant, mysterious predators known as the Titans. Three mighty walls—Wall Maria, Rose, and Sheena—provided peace and protection for humanity for over a hundred years. That peace, however, was shattered when the Colossus Titan and Armored Titan appeared and d...
On his first day of junior high, Eren Yeager comes face-to-face with a titan—and has his lunch stolen! From that day on, he holds a grudge against titans for taking his favorite food from him, a cheeseburger, vowing to eliminate their kind once and for all.

In [194]:
# SCORED_BY
# from data_modeling EXAMPLE 1 (Cosine, un-normalized), ['Attack on Titan'] input,
# ['Sword Art Online', 'Dragon Ball Z', 'Code Geass: Lelouch of the Rebellion R2', 
#  'Death Note', 'One Punch Man'] output
in_list = ['Attack on Titan', 'Attack on Titan: Since That Day', 'Attack on Titan: Crimson Bow and Arrow',
           'Attack on Titan: Wings of Freedom', 'Attack on Titan Season 2', 'Attack on Titan: Junior High',
           'Attack on Titan Season 3']
insyn_list = []
for t in out_list:
    insyn_list.append(list(gen_df[gen_df['title_english'] == t]['scored_by'])[0])
print(insyn_list)
out_list = ['Miss Kobayashi\'s Dragon Maid', 'Rosario + Vampire', 
            'My Teen Romantic Comedy SNAFU TOO!', 'GATE',
            'JoJo\'s Bizarre Adventure']
outsyn_list = []
for t in out_list:
    outsyn_list.append(list(gen_df[gen_df['title_english'] == t]['scored_by'])[0])
sumz = 0
for syn in outsyn_list:
    z = (std_z(syn, avg(insyn_list), len(outsyn_list)))
    sumz += abs(z)
    print(">> \t%s, \t%s"%(syn,z))

print("AVG ST.Z: %s"%(sumz/len(outsyn_list)))
print("AVG SQ ST.Z: %s"%(avg_std_z(outsyn_list,avg(insyn_list))))

[244021, 245586, 251353, 235214, 14356]
>> 	244021, 	9183.0
>> 	245586, 	9496.0
>> 	251353, 	10649.4
>> 	235214, 	7421.6
>> 	14356, 	-36750.0
AVG ST.Z: 14700.0
AVG SQ ST.Z: 92020.48336973676


In [195]:
# POPULARITY
# from data_modeling EXAMPLE 1 (Cosine, un-normalized), ['Attack on Titan'] input,
# ['Sword Art Online', 'Dragon Ball Z', 'Code Geass: Lelouch of the Rebellion R2', 
#  'Death Note', 'One Punch Man'] output
in_list = ['Attack on Titan', 'Attack on Titan: Since That Day', 'Attack on Titan: Crimson Bow and Arrow',
           'Attack on Titan: Wings of Freedom', 'Attack on Titan Season 2', 'Attack on Titan: Junior High',
           'Attack on Titan Season 3']
insyn_list = []
for t in out_list:
    insyn_list.append(list(gen_df[gen_df['title_english'] == t]['popularity'])[0])
print(insyn_list)
out_list = ['Miss Kobayashi\'s Dragon Maid', 'Rosario + Vampire', 
            'My Teen Romantic Comedy SNAFU TOO!', 'GATE',
            'JoJo\'s Bizarre Adventure']
outsyn_list = []
for t in out_list:
    outsyn_list.append(list(gen_df[gen_df['title_english'] == t]['popularity'])[0])
sumz = 0
for syn in outsyn_list:
    z = (std_z(syn, avg(insyn_list), len(outsyn_list)))
    sumz += abs(z)
    print(">> \t%s, \t%s"%(syn,z))

print("AVG ST.Z: %s"%(sumz/len(outsyn_list)))
print("AVG SQ ST.Z: %s"%(avg_std_z(outsyn_list,avg(insyn_list))))

[134, 143, 148, 139, 2139]
>> 	134, 	-81.32000000000001
>> 	143, 	-79.52000000000001
>> 	148, 	-78.52000000000001
>> 	139, 	-80.32000000000001
>> 	2139, 	319.68
AVG ST.Z: 127.87200000000003
AVG SQ ST.Z: 799.2132631532087


In [196]:
# EPISODES
# from data_modeling EXAMPLE 1 (Cosine, un-normalized), ['Attack on Titan'] input,
# ['Sword Art Online', 'Dragon Ball Z', 'Code Geass: Lelouch of the Rebellion R2', 
#  'Death Note', 'One Punch Man'] output
in_list = ['Attack on Titan', 'Attack on Titan: Since That Day', 'Attack on Titan: Crimson Bow and Arrow',
           'Attack on Titan: Wings of Freedom', 'Attack on Titan Season 2', 'Attack on Titan: Junior High',
           'Attack on Titan Season 3']
insyn_list = []
for t in out_list:
    insyn_list.append(list(gen_df[gen_df['title_english'] == t]['episodes'])[0])
print(insyn_list)
out_list = ['Miss Kobayashi\'s Dragon Maid', 'Rosario + Vampire', 
            'My Teen Romantic Comedy SNAFU TOO!', 'GATE',
            'JoJo\'s Bizarre Adventure']
outsyn_list = []
for t in out_list:
    outsyn_list.append(list(gen_df[gen_df['title_english'] == t]['episodes'])[0])
sumz = 0
for syn in outsyn_list:
    z = (std_z(syn, avg(insyn_list), len(outsyn_list)))
    sumz += abs(z)
    print(">> \t%s, \t%s"%(syn,z))

print("AVG ST.Z: %s"%(sumz/len(outsyn_list)))
print("AVG SQ ST.Z: %s"%(avg_std_z(outsyn_list,avg(insyn_list))))

[13.0, 13.0, 13.0, 12.0, 7.0]
>> 	13.0, 	0.2800000000000001
>> 	13.0, 	0.2800000000000001
>> 	13.0, 	0.2800000000000001
>> 	12.0, 	0.08000000000000007
>> 	7.0, 	-0.9199999999999999
AVG ST.Z: 0.36800000000000005
AVG SQ ST.Z: 2.33238075793812


In [197]:
# RANK
# from data_modeling EXAMPLE 1 (Cosine, un-normalized), ['Attack on Titan'] input,
# ['Sword Art Online', 'Dragon Ball Z', 'Code Geass: Lelouch of the Rebellion R2', 
#  'Death Note', 'One Punch Man'] output
in_list = ['Attack on Titan', 'Attack on Titan: Since That Day', 'Attack on Titan: Crimson Bow and Arrow',
           'Attack on Titan: Wings of Freedom', 'Attack on Titan Season 2', 'Attack on Titan: Junior High',
           'Attack on Titan Season 3']
insyn_list = []
for t in out_list:
    insyn_list.append(list(gen_df[gen_df['title_english'] == t]['rank'])[0])
print(insyn_list)
out_list = ['Miss Kobayashi\'s Dragon Maid', 'Rosario + Vampire', 
            'My Teen Romantic Comedy SNAFU TOO!', 'GATE',
            'JoJo\'s Bizarre Adventure']
outsyn_list = []
for t in out_list:
    outsyn_list.append(list(gen_df[gen_df['title_english'] == t]['rank'])[0])
sumz = 0
for syn in outsyn_list:
    z = (std_z(syn, avg(insyn_list), len(outsyn_list)))
    sumz += abs(z)
    print(">> \t%s, \t%s"%(syn,z))

print("AVG ST.Z: %s"%(sumz/len(outsyn_list)))
print("AVG SQ ST.Z: %s"%(avg_std_z(outsyn_list,avg(insyn_list))))

[359, 3569, 271, 758, 2818]
>> 	359, 	-239.2
>> 	3569, 	402.8
>> 	271, 	-256.8
>> 	758, 	-159.4
>> 	2818, 	252.6
AVG ST.Z: 262.15999999999997
AVG SQ ST.Z: 1368.6231037067876


In [198]:
# MEMBERS
# from data_modeling EXAMPLE 1 (Cosine, un-normalized), ['Attack on Titan'] input,
# ['Sword Art Online', 'Dragon Ball Z', 'Code Geass: Lelouch of the Rebellion R2', 
#  'Death Note', 'One Punch Man'] output
in_list = ['Attack on Titan', 'Attack on Titan: Since That Day', 'Attack on Titan: Crimson Bow and Arrow',
           'Attack on Titan: Wings of Freedom', 'Attack on Titan Season 2', 'Attack on Titan: Junior High',
           'Attack on Titan Season 3']
insyn_list = []
for t in out_list:
    insyn_list.append(list(gen_df[gen_df['title_english'] == t]['members'])[0])
print(insyn_list)
out_list = ['Miss Kobayashi\'s Dragon Maid', 'Rosario + Vampire', 
            'My Teen Romantic Comedy SNAFU TOO!', 'GATE',
            'JoJo\'s Bizarre Adventure']
outsyn_list = []
for t in out_list:
    outsyn_list.append(list(gen_df[gen_df['title_english'] == t]['members'])[0])
sumz = 0
for syn in outsyn_list:
    z = (std_z(syn, avg(insyn_list), len(outsyn_list)))
    sumz += abs(z)
    print(">> \t%s, \t%s"%(syn,z))

print("AVG ST.Z: %s"%(sumz/len(outsyn_list)))
print("AVG SQ ST.Z: %s"%(avg_std_z(outsyn_list,avg(insyn_list))))

[426274, 414882, 405743, 419943, 34055]
>> 	426274, 	17218.919999999995
>> 	414882, 	14940.519999999995
>> 	405743, 	13112.719999999996
>> 	419943, 	15952.719999999996
>> 	34055, 	-61224.880000000005
AVG ST.Z: 24489.951999999997
AVG SQ ST.Z: 153209.48038629984


In [199]:
# FAVORITES
# from data_modeling EXAMPLE 1 (Cosine, un-normalized), ['Attack on Titan'] input,
# ['Sword Art Online', 'Dragon Ball Z', 'Code Geass: Lelouch of the Rebellion R2', 
#  'Death Note', 'One Punch Man'] output
in_list = ['Attack on Titan', 'Attack on Titan: Since That Day', 'Attack on Titan: Crimson Bow and Arrow',
           'Attack on Titan: Wings of Freedom', 'Attack on Titan Season 2', 'Attack on Titan: Junior High',
           'Attack on Titan Season 3']
insyn_list = []
for t in out_list:
    insyn_list.append(list(gen_df[gen_df['title_english'] == t]['favorites'])[0])
print(insyn_list)
out_list = ['Miss Kobayashi\'s Dragon Maid', 'Rosario + Vampire', 
            'My Teen Romantic Comedy SNAFU TOO!', 'GATE',
            'JoJo\'s Bizarre Adventure']
outsyn_list = []
for t in out_list:
    outsyn_list.append(list(gen_df[gen_df['title_english'] == t]['favorites'])[0])
sumz = 0
for syn in outsyn_list:
    z = (std_z(syn, avg(insyn_list), len(outsyn_list)))
    sumz += abs(z)
    print(">> \t%s, \t%s"%(syn,z))

print("AVG ST.Z: %s"%(sumz/len(outsyn_list)))
print("AVG SQ ST.Z: %s"%(avg_std_z(outsyn_list,avg(insyn_list))))

[7684, 3262, 10758, 4266, 166]
>> 	7684, 	491.36
>> 	3262, 	-393.03999999999996
>> 	10758, 	1106.16
>> 	4266, 	-192.23999999999995
>> 	166, 	-1012.24
AVG ST.Z: 639.008
AVG SQ ST.Z: 3661.346113111952
